In [1]:
import requests
import subprocess
import sys
import os
import tarfile

In [2]:
def get_top_pypi_packages(url='https://hugovk.github.io/top-pypi-packages/top-pypi-packages-30-days.json'):
    response = requests.get(url)
    response.raise_for_status() 
    
    data = response.json()
    packages = [package['project'] for package in data['rows'][:1000]]

    return packages

top_packages = get_top_pypi_packages()
print(top_packages)


['boto3', 'botocore', 'urllib3', 'requests', 'wheel', 'certifi', 'idna', 'typing-extensions', 'charset-normalizer', 'pip', 'setuptools', 'python-dateutil', 's3transfer', 'packaging', 'aiobotocore', 'pyyaml', 'six', 's3fs', 'fsspec', 'numpy', 'cryptography', 'grpcio-status', 'google-api-core', 'cffi', 'pycparser', 'pypular', 'pandas', 'importlib-metadata', 'pyasn1', 'rsa', 'zipp', 'attrs', 'click', 'pydantic', 'protobuf', 'jmespath', 'platformdirs', 'pytz', 'jinja2', 'colorama', 'markupsafe', 'pyjwt', 'awscli', 'tomli', 'wrapt', 'google-auth', 'googleapis-common-protos', 'filelock', 'cachetools', 'requests-oauthlib', 'oauthlib', 'pluggy', 'virtualenv', 'pyarrow', 'docutils', 'jsonschema', 'pyasn1-modules', 'pytest', 'exceptiongroup', 'aiohttp', 'pyparsing', 'sqlalchemy', 'scipy', 'isodate', 'multidict', 'psutil', 'pyopenssl', 'yarl', 'iniconfig', 'decorator', 'soupsieve', 'pygments', 'tzdata', 'async-timeout', 'beautifulsoup4', 'frozenlist', 'aiosignal', 'tqdm', 'grpcio', 'pillow', 'req

In [35]:
def dl_packages(packages, target_directory):
    for package in packages:
        package_dir = os.path.join(target_directory, package)
        os.makedirs(package_dir, exist_ok=True)
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "download", package, "--no-binary=:all:", "--no-deps", "--dest", package_dir])
            print(f"Downloaded {package} into {package_dir}")
        except subprocess.CalledProcessError as e:
            print(f"Failed to download {package}: {e}")

top_packages = get_top_pypi_packages()

download_directory = '/mnt/volume_nyc1_01/benignPyPI/'

dl_packages(top_packages, download_directory)

  Using cached boto3-1.34.136.tar.gz (108 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Saved /mnt/volume_nyc1_01/benignPyPI/boto3/boto3-1.34.136.tar.gz
Successfully downloaded boto3
Downloaded boto3 into /mnt/volume_nyc1_01/benignPyPI/boto3
  Using cached botocore-1.34.136.tar.gz (12.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Saved /mnt/volume_nyc1_01/benignPyPI/botocore/botocore-1.34.136.tar.gz
Successfully downlo

In [36]:
repo_dir = "/mnt/volume_nyc1_01/benignPyPI/"
#Find all tar.gz files in the repository
tar_files = []
for root, dirs, files in os.walk(repo_dir):
    for file in files:
        if file.endswith(".tar.gz") or file.endswith(".tar.bz2") or file.endswith(".tar.xz"):
            tar_files.append(os.path.join(root, file))

#Unzip the files with different compression methods
for tar_file in tar_files:
    if tar_file.endswith(".tar.gz"):
        mode = 'r:gz'
    elif tar_file.endswith(".tar.bz2"):
        mode = 'r:bz2'
    elif tar_file.endswith(".tar.xz"):
        mode = 'r:xz'
    else:
        mode = 'r'
    
    try:
        with tarfile.open(tar_file, mode) as tar_ref:
            extract_dir = os.path.splitext(os.path.splitext(tar_file)[0])[0]
            tar_ref.extractall(extract_dir)
            print(f"Extracted {tar_file} to {extract_dir}")
    except (tarfile.ReadError, tarfile.CompressionError) as e:
        print(f"Failed to extract {tar_file}: {e}")

print("Extraction process completed.")

Extracted /mnt/volume_nyc1_01/benignPyPI/botocore/botocore-1.34.136.tar.gz to /mnt/volume_nyc1_01/benignPyPI/botocore/botocore-1.34.136
Extracted /mnt/volume_nyc1_01/benignPyPI/requests/requests-2.32.3.tar.gz to /mnt/volume_nyc1_01/benignPyPI/requests/requests-2.32.3
Extracted /mnt/volume_nyc1_01/benignPyPI/urllib3/urllib3-2.2.2.tar.gz to /mnt/volume_nyc1_01/benignPyPI/urllib3/urllib3-2.2.2
Extracted /mnt/volume_nyc1_01/benignPyPI/boto3/boto3-1.34.136.tar.gz to /mnt/volume_nyc1_01/benignPyPI/boto3/boto3-1.34.136
Extracted /mnt/volume_nyc1_01/benignPyPI/wheel/wheel-0.43.0.tar.gz to /mnt/volume_nyc1_01/benignPyPI/wheel/wheel-0.43.0
Extraction process completed.
